In [ ]:
## import libraries
import os, sys
import yaml
import re
import xarray as xr
import pandas as pd
import numpy as np
from datetime import timedelta
%matplotlib inline
from datetime import timedelta

# plotting
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.gridspec import GridSpec
from matplotlib.colorbar import Colorbar # different way to handle colorbar
import matplotlib.patches as mpatches
import textwrap
import cmocean.cm as cmo

# cartopy
import cartopy.crs as ccrs
from mpl_toolkits.axes_grid1 import AxesGrid
from cartopy.mpl.geoaxes import GeoAxes
import cartopy.feature as cfeature

# import personal modules
sys.path.append('../modules')
import custom_cmaps as ccmap
from plotter import draw_basemap
import ar_funcs
import mclimate_funcs as mclim_func
# dask.config.set(**{'array.slicing.split_large_chunks': True})

In [ ]:
path_to_data = '/expanse/nfs/cw3e/cwp140/'      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [ ]:
varname = 'freezing_level'
date = '20051118'
F = 72

valid_time = pd.to_datetime(str(date)) # this is the AR date
d = valid_time - timedelta(hours=F) ## we want the initialization date to be x hours prior to AR date
mon = d.strftime('%m') # month of initialization
day = d.strftime('%d') # day of initialization
init_date = d.strftime('%Y%m%d')

## load the reforecast for the initialization date
fc = mclim_func.load_reforecast(init_date, varname)
fc = fc.sel(step=F)
fc = fc.load()

## load the mclimate for the same initialization date
mclimate = mclim_func.load_mclimate(mon, day, varname)
mclimate = mclimate.sel(step=F)
mclimate = mclimate.isel(dayofyear=0)
mclimate = mclimate.load()

## compare the mclimate to the reforecast
ds = mclim_func.compare_mclimate_to_reforecast(fc, mclimate, varname)
## add time to ds
ds = ds.assign_coords({"init_time": d, "valid_time": valid_time})
ds

In [ ]:
fc_lst = [fc]
ds_lst = [ds]

# Set up projection
mapcrs = ccrs.Mercator()
# mapcrs = ccrs.PlateCarree()
datacrs = ccrs.PlateCarree()

# Set tick/grid locations
lats = fc.lat.values
lons = fc.lon.values
dx = np.arange(lons.min().round(),lons.max().round()+10,10)
dy = np.arange(lats.min().round(),lats.max().round()+5,5)

ext = [-170., -120., 40., 65.]

In [ ]:
# Create figure
fig = plt.figure(figsize=(8, 5))
fig.dpi = 300
fname = path_to_figs + 'freezing_level_mclimate_test'
fmt = 'png'

nrows = 1
ncols = 2

# contour labels
kw_clabels = {'fontsize': 7, 'inline': True, 'inline_spacing': 7, 'fmt': '%i',
              'rightside_up': True, 'use_clabeltext': True}

kw_ticklabels = {'size': 10, 'color': 'dimgray', 'weight': 'light'}

## Use gridspec to set up a plot with a series of subplots that is
## n-rows by n-columns
gs = GridSpec(nrows, ncols, height_ratios=[1], width_ratios = [1, 0.05], wspace=0.001, hspace=0.05)
## use gs[rows index, columns index] to access grids

###################
### CLIMATOLOGY ###
###################
leftlats_lst = [True, False]
for i, (fc, ds) in enumerate(zip(fc_lst, ds_lst)):
    ax = fig.add_subplot(gs[0, i], projection=mapcrs) 
    ax = draw_basemap(ax, extent=ext, xticks=dx, yticks=dy, left_lats=leftlats_lst[i], right_lats=False, bottom_lons=True)
    
    # Contour Filled
    data = ds.mclimate.values*100.
    cmap, norm, bnds = ccmap.cmap('mclimate_purple')
    cf = ax.contourf(ds.lon, ds.lat, data, transform=datacrs,
                     levels=bnds, cmap=cmap, norm=norm, alpha=0.9, extend='neither')
    
    # Contour Lines
    clevs = np.arange(0., 4500., 500.)
    cs = ax.contour(fc.lon, fc.lat, fc[varname].values, transform=datacrs,
                     levels=clevs, colors='k',
                     linewidths=0.75, linestyles='solid')
    plt.clabel(cs, **kw_clabels)
    
    ts = pd.to_datetime(str(ds.init_time.values)) 
    init_time = ts.strftime('%HZ %d %b %Y')
    start_date = ts - timedelta(days=45)
    start_date = start_date.strftime('%d-%b')
    end_date = ts + timedelta(days=45)
    end_date = end_date.strftime('%d-%b')
    
    ts = pd.to_datetime(str(ds.valid_time.values)) 
    valid_time = ts.strftime('%HZ %d %b %Y')
    
    ax.set_title('Model Run: {0}'.format(init_time), loc='left', fontsize=10)
    ax.set_title('Valid Date: {0}'.format(valid_time), loc='right', fontsize=10)

    
    txt = 'Relative to all 162-h GEFSv12 reforecasts initialized between {0} and {1} (2000-2019)'.format(start_date, end_date)
    ann_ax = fig.add_subplot(gs[-1, i])
    ann_ax.axis('off')
    ann_ax.annotate(textwrap.fill(txt, 60), # this is the text
               (0, 0.), # these are the coordinates to position the label
                textcoords="offset points", # how to position the text
                xytext=(25,-35), # distance from text to points (x,y)
                ha='left', # horizontal alignment can be left, right or center
                **kw_ticklabels)


# Add color bar
cbax = plt.subplot(gs[0,-1]) # colorbar axis
cb = Colorbar(ax = cbax, mappable = cf, orientation = 'vertical', ticklocation = 'right')
cb.set_label('Model Climate Percentile Rank (xth)', fontsize=10)
cb.ax.tick_params(labelsize=8)

fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi)

# Show
plt.show()